In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [5]:
## Read data from url 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_url = requests.get(url)
toronto_url

<Response [200]>

In [6]:
toronto_data = pd.read_html(toronto_url.text)
len(toronto_data), type(toronto_data)

(3, list)

In [7]:
## Refer to first table
toronto_data = toronto_data[0]
toronto_data.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
## Ignore rows with Not assigned borough
indexBorough = toronto_data[toronto_data['Borough'] == 'Not assigned'].index
toronto_data.drop(indexBorough, inplace=True)

In [9]:
## Neighborhoods separated with a comma
toronto_data = toronto_data.replace('/',', ',regex=True)

In [10]:
## Neighborhood will be the same as the borough if Not assigned
toronto_data.Neighborhood.replace('Not assigned',toronto_data.Borough,inplace=True)

In [11]:
toronto_data.tail()

,Postal code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea..."
178,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bl..."


In [9]:
toronto_data.shape

(103, 3)

In [16]:
Loc_data = pd.read_csv('http://cocl.us/Geospatial_data')

In [17]:
Loc_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
Loc_data.rename(columns={'Postal Code':'Postal code'}, inplace=True)

In [19]:
Loc_data.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
cols = ['Postal code']
toronto_data.join(Loc_data.set_index(cols), on=cols)

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [21]:
toronto_data.reset_index()

,index,Postal code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,5,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,8,M9A,Etobicoke,Islington Avenue
6,9,M1B,Scarborough,"Malvern , Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"
